**'Fraud or Not' ML Model**


In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
dataset = pd.read_csv('/content/drive/MyDrive/Fraud.csv') 

In [5]:
dataset.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


**Features like 'step' , 'nameOrig' and 'nameDest' are of no use for us as they dont carry any value for our predictions. nameOrig and nameDest will be unique for each new costumer and hence we cannot find a corelation between these variables and our dependent variables. In 'step' variable, we wont get much of a corelation hence i've excluded it as well.**

In [6]:
X = dataset.iloc[:,1:3] 
Y = dataset.iloc[:,4:6]
Z = dataset.iloc[:,7:9]

print(X)
print('\n')

print(Y) 
print('\n')
      
print(Z) 
print('\n')      

             type      amount
0         PAYMENT     9839.64
1         PAYMENT     1864.28
2        TRANSFER      181.00
3        CASH_OUT      181.00
4         PAYMENT    11668.14
...           ...         ...
6362615  CASH_OUT   339682.13
6362616  TRANSFER  6311409.28
6362617  CASH_OUT  6311409.28
6362618  TRANSFER   850002.52
6362619  CASH_OUT   850002.52

[6362620 rows x 2 columns]


         oldbalanceOrg  newbalanceOrig
0            170136.00       160296.36
1             21249.00        19384.72
2               181.00            0.00
3               181.00            0.00
4             41554.00        29885.86
...                ...             ...
6362615      339682.13            0.00
6362616     6311409.28            0.00
6362617     6311409.28            0.00
6362618      850002.52            0.00
6362619      850002.52            0.00

[6362620 rows x 2 columns]


         oldbalanceDest  newbalanceDest
0                  0.00            0.00
1                  0.00         

In [7]:
Final_dataset = np.concatenate((X,Y,Z),1)
Final_dataset

array([['PAYMENT', 9839.64, 170136.0, 160296.36, 0.0, 0.0],
       ['PAYMENT', 1864.28, 21249.0, 19384.72, 0.0, 0.0],
       ['TRANSFER', 181.0, 181.0, 0.0, 0.0, 0.0],
       ...,
       ['CASH_OUT', 6311409.28, 6311409.28, 0.0, 68488.84, 6379898.11],
       ['TRANSFER', 850002.52, 850002.52, 0.0, 0.0, 0.0],
       ['CASH_OUT', 850002.52, 850002.52, 0.0, 6510099.11, 7360101.63]],
      dtype=object)

**OneHotEncoding the 'type' column**

In [8]:
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import OneHotEncoder 
ct = ColumnTransformer(transformers = [('encoder',OneHotEncoder(),[0])], remainder = 'passthrough') 
Final_dataset = np.array(ct.fit_transform(Final_dataset))
print(Final_dataset)

[[0.0 0.0 0.0 ... 160296.36 0.0 0.0]
 [0.0 0.0 0.0 ... 19384.72 0.0 0.0]
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]
 ...
 [0.0 1.0 0.0 ... 0.0 68488.84 6379898.11]
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]
 [0.0 1.0 0.0 ... 0.0 6510099.11 7360101.63]]


In [9]:
Final_dataset.shape

(6362620, 10)

In [21]:
Final_dataset[:,:5]

array([[0.0, 0.0, 0.0, 1.0, 0.0],
       [0.0, 0.0, 0.0, 1.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 1.0],
       ...,
       [0.0, 1.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 1.0],
       [0.0, 1.0, 0.0, 0.0, 0.0]], dtype=object)

**Replacing the zero values or unknown values with the mean of that feature column**

In [10]:
from sklearn.impute import SimpleImputer
alpha = SimpleImputer(missing_values=0.0,strategy='mean')
Final_dataset[:,8:] = alpha.fit_transform(Final_dataset[:,8:])
print(Final_dataset)
print('\n')

[[0.0 0.0 0.0 ... 160296.36 1914407.40702929 1986697.698358111]
 [0.0 0.0 0.0 ... 19384.72 1914407.40702929 1986697.698358111]
 [0.0 0.0 0.0 ... 0.0 1914407.40702929 1986697.698358111]
 ...
 [0.0 1.0 0.0 ... 0.0 68488.84 6379898.11]
 [0.0 0.0 0.0 ... 0.0 1914407.40702929 1986697.698358111]
 [0.0 1.0 0.0 ... 0.0 6510099.11 7360101.63]]




In [11]:
Final_dataset[:,8:]

array([[1914407.40702929, 1986697.698358111],
       [1914407.40702929, 1986697.698358111],
       [1914407.40702929, 1986697.698358111],
       ...,
       [68488.84, 6379898.11],
       [1914407.40702929, 1986697.698358111],
       [6510099.11, 7360101.63]], dtype=object)

In [12]:
Final_dataset.shape

(6362620, 10)

In [13]:
fraud = dataset.iloc[:,-2].values
fraud

array([0, 0, 1, ..., 1, 1, 1])

In [14]:
len(fraud)

6362620

In [15]:
from sklearn.model_selection import train_test_split

**Splitting Train-Test as 80-20 split**

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(Final_dataset, fraud, test_size = 0.2, random_state = 1)

In [17]:
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(5090096, 10) (1272524, 10) (5090096,) (1272524,)


**Feature scalling is important here as some values are way too higher as compared to others**

In [23]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:, 5:] = sc.fit_transform(X_train[:, 5:]) 
print(X_train)
print('\n')

X_test[:, 5:] = sc.transform(X_test[:, 5:]) 
print(X_test)
print('\n')

[[0.0 0.0 0.0 ... -0.2923588746637241 -5.340973615521863e-05
  -0.00010504984980609933]
 [0.0 0.0 0.0 ... -0.2923588746637241 -5.340973615521863e-05
  -0.00010504984980609933]
 [0.0 1.0 0.0 ... -0.2923588746637241 -5.340973615521863e-05
  -0.5166045833511143]
 ...
 [0.0 1.0 0.0 ... -0.2923588746637241 2.0356892127714126
  1.8897365399791413]
 [0.0 0.0 0.0 ... -0.2923588746637241 -5.340973615521863e-05
  -0.00010504984980609933]
 [0.0 0.0 0.0 ... -0.2923588746637241 -5.340973615521863e-05
  -0.00010504984980609933]]


[[1.0 0.0 0.0 ... -0.2815455451692375 -0.5347865038592392
  -0.5194191460674986]
 [0.0 0.0 0.0 ... -0.2923588746637241 -5.340973615521863e-05
  -0.00010504984980609933]
 [0.0 0.0 0.0 ... -0.2923588746637241 -5.340973615521863e-05
  -0.00010504984980609933]
 ...
 [0.0 1.0 0.0 ... -0.2923588746637241 -0.40236876318987475
  -0.29105481154055696]
 [0.0 0.0 0.0 ... -0.18182845740189166 -5.340973615521863e-05
  -0.00010504984980609933]
 [0.0 0.0 0.0 ... -0.2923588746637241 -5.34

**Lets use Logistic Regression for this classfication**

In [24]:
from sklearn.linear_model import LogisticRegression

In [25]:
classifier = LogisticRegression(random_state=1)

In [26]:
classifier.fit(X_train,Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=1)

In [30]:
Y_pred = classifier.predict(X_test)

In [31]:
Y_pred

array([0, 0, 0, ..., 0, 0, 0])

**Here, the output below shows predictions in column 1 and the original test results in column 2** 

In [40]:
print(np.concatenate((Y_pred.reshape(len(Y_pred),1), Y_test.reshape(len(Y_test),1)),1))

[[0 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


**Checking the Confusion matrix and the Accuracy score** 

In [47]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(Y_test, Y_pred)
print(cm)
print('\n')
print("accuracy score = ",accuracy_score(Y_test, Y_pred))

[[1270802      75]
 [   1049     598]]


accuracy score =  0.9991167160697951


**Lets now build a model for the flagged fraud alerts**

In [48]:
flagged_fraud = dataset.iloc[:,-1].values

In [49]:
X_train_dash, X_test_dash, Y_train_dash, Y_test_dash = train_test_split(Final_dataset, flagged_fraud, test_size = 0.2, random_state = 1)

In [50]:
print(X_train_dash.shape, X_test_dash.shape, Y_train_dash.shape, Y_test_dash.shape)

(5090096, 10) (1272524, 10) (5090096,) (1272524,)


In [51]:
classifier.fit(X_train_dash,Y_train_dash)

LogisticRegression(random_state=1)

In [52]:
Y_pred_dash = classifier.predict(X_test_dash)

In [53]:
cm_dash = confusion_matrix(Y_test_dash, Y_pred_dash)
print(cm_dash)
print('\n')
print("accuracy score for flagged fraud = ",accuracy_score(Y_test_dash, Y_pred_dash))

[[1272503      20]
 [      1       0]]


accuracy score for flagged fraud =  0.9999834973642934


When a company updates its infrastructure, It must be considered to maintain a dataset of each and every customer taking the services so that we have a record of whats going on in the company. Later on, we can run several ML models to check whether everything if running well and we can even predict what kind of future improvements we need to do for the company. In short, Insights are very important and for that, a well maintained dataset is needed.